In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cifar-10/trainLabels.csv
/kaggle/input/cifar-10/sampleSubmission.csv
/kaggle/input/cifar-10/test.7z
/kaggle/input/cifar-10/train.7z


In [2]:
import pandas as pd
from glob import glob
import keras
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.applications.xception import Xception
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten
from keras.preprocessing.image import ImageDataGenerator
import cv2
import os
%matplotlib inline
import matplotlib.pyplot as plt
from collections import Counter
from keras.layers import *
from keras.models import *
from keras.applications import *
from sklearn.manifold import TSNE
from keras.optimizers import *
from keras.regularizers import *
from sklearn.preprocessing import LabelEncoder

In [3]:
x_train = None
y_train = None
x_test = None
labels_path = '/kaggle/input/cifar-10/trainLabels.csv'
sample_path = 'data/sampleSubmission.csv'
train_path = '/kaggle/input/cifar-10/train.7z'
test_path = '/kaggle/input/cifar-10/test.7z'

In [4]:
labels = pd.read_csv(labels_path)
print("First few elements : ")
labels.head(5)


First few elements : 


,id,label
0,1,frog
1,2,truck
2,3,truck
3,4,deer
4,5,automobile


In [5]:
unique_labels = sorted(set(labels['label']))
n_class = len(unique_labels)
class_to_num = dict(zip(unique_labels, range(n_class)))
num_to_class = dict(zip(range(n_class), unique_labels))


print("Dimensions of the labels dataframe : ", labels.shape)
print('The',n_class,'different labels are :\n', unique_labels)
print('\nThe distribution of the classes are :\n', Counter(labels['label']))

Dimensions of the labels dataframe :  (50000, 2)
The 10 different labels are :
 ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

The distribution of the classes are :
 Counter({'frog': 5000, 'truck': 5000, 'deer': 5000, 'automobile': 5000, 'bird': 5000, 'horse': 5000, 'ship': 5000, 'cat': 5000, 'dog': 5000, 'airplane': 5000})


In [6]:
id = labels['id']
encoded = pd.get_dummies(labels['label'])
print('\nThe new dimensions of the dataframe are :',encoded.shape,'\n\n')
encoded.head(4)


The new dimensions of the dataframe are : (50000, 10) 




,airplane,automobile,bird,cat,deer,dog,frog,horse,ship,truck
0,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,1,0,0,0,0,0


In [7]:
encoded = np.asarray(encoded)
id = np.asarray(id)
print('The dimensions of the one hot encoded labels numpy array are :',np.shape(encoded))
print('The dimensions of the id numpy array are :',np.shape(id))

The dimensions of the one hot encoded labels numpy array are : (50000, 10)
The dimensions of the id numpy array are : (50000,)


In [15]:
import tensorflow as tf
# classfying cifar10 dataset using resnet 50
from tensorflow.keras import datasets, layers, models

def preprocess_image_input(input_images):
  input_images = input_images.astype('float32')
  output_ims = tf.keras.applications.resnet50.preprocess_input(input_images)
  return output_ims




base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
base_model.trainable = False
xinp = layers.Input(shape=(32, 32, 3))
x = base_model(xinp, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(1024, activation="relu")(x)
x = tf.keras.layers.Dense(512, activation="relu")(x)
x = layers.Dense(10, activation='softmax')(x)
model = tf.keras.Model(xinp, x)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()
x_train, x_test = preprocess_image_input(x_train),preprocess_image_input(x_test) 
model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))
model.evaluate(x_test, y_test)
 

Epoch 1/10
1563/1563 [==============================] - 32s 18ms/step - loss: 1.1532 - accuracy: 0.6072 - val_loss: 1.0277 - val_accuracy: 0.6379
Epoch 2/10
1563/1563 [==============================] - 26s 16ms/step - loss: 0.9100 - accuracy: 0.6811 - val_loss: 0.9780 - val_accuracy: 0.6590
Epoch 3/10
1563/1563 [==============================] - 26s 16ms/step - loss: 0.7998 - accuracy: 0.7184 - val_loss: 0.9829 - val_accuracy: 0.6622
Epoch 4/10
1563/1563 [==============================] - 25s 16ms/step - loss: 0.6899 - accuracy: 0.7552 - val_loss: 1.0140 - val_accuracy: 0.6718
Epoch 5/10
1563/1563 [==============================] - 26s 16ms/step - loss: 0.5856 - accuracy: 0.7914 - val_loss: 1.0984 - val_accuracy: 0.6697
Epoch 6/10
1563/1563 [==============================] - 27s 17ms/step - loss: 0.4832 - accuracy: 0.8261 - val_loss: 1.2540 - val_accuracy: 0.6600
Epoch 7/10
1563/1563 [==============================] - 26s 17ms/step - loss: 0.3886 - accuracy: 0.8608 - val_loss: 1.4092 -

[1.769891381263733, 0.6606000065803528]

In [10]:
import tensorflow as tf
# classfying cifar10 dataset using resnet 50
from tensorflow.keras import datasets, layers, models

base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
base_model.trainable = False
xinp = layers.Input(shape=(32, 32, 3))
x = base_model(xinp, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(10, activation='softmax')(x)
model = tf.keras.Model(xinp, x)
blr = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=blr),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
# history = model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))
# model.evaluate(x_test, y_test)
model.summary()
 

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 resnet50 (Functional)       (None, 1, 1, 2048)        23587712  
                                                                 
 global_average_pooling2d_2   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_5 (Dense)             (None, 512)               1049088   
                                                                 
 dropout_3 (Dropout)         (None, 512)               0         
                                                                 
 dense_6 (Dense)             (None, 256)               131328    
                                                           

In [11]:
history = model.fit(x_train, y_train, epochs=100, validation_data=(x_test, y_test), batch_size = 100)
model.evaluate(x_test, y_test)

Epoch 1/100


/opt/conda/lib/python3.7/site-packages/keras/backend.py:5586: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits, "Softmax", "sparse_categorical_crossentropy"


500/500 [==============================] - 16s 23ms/step - loss: 2.2886 - accuracy: 0.1391 - val_loss: 2.1060 - val_accuracy: 0.2305
Epoch 2/100
500/500 [==============================] - 11s 22ms/step - loss: 2.0693 - accuracy: 0.2310 - val_loss: 1.9452 - val_accuracy: 0.2895
Epoch 3/100
500/500 [==============================] - 11s 21ms/step - loss: 1.9746 - accuracy: 0.2716 - val_loss: 1.8774 - val_accuracy: 0.3123
Epoch 4/100
500/500 [==============================] - 11s 22ms/step - loss: 1.9207 - accuracy: 0.2979 - val_loss: 1.8233 - val_accuracy: 0.3448
Epoch 5/100
500/500 [==============================] - 10s 19ms/step - loss: 1.8766 - accuracy: 0.3167 - val_loss: 1.7854 - val_accuracy: 0.3619
Epoch 6/100
500/500 [==============================] - 10s 20ms/step - loss: 1.8403 - accuracy: 0.3292 - val_loss: 1.7748 - val_accuracy: 0.3669
Epoch 7/100
500/500 [==============================] - 11s 22ms/step - loss: 1.8190 - accuracy: 0.3397 - val_loss: 1.7607 - val_accuracy: 0.36

[1.4671714305877686, 0.4715999960899353]

In [17]:
# now unfreeze some layers and train after that
base_model.trainable = True
print("Number of layers in the base model: ", len(base_model.layers))
fine_tune_at = 150
blr=0.0001
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable =  False
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer = 'SGD',
              metrics=['accuracy'])


Number of layers in the base model:  175


In [18]:
fine_tune_epochs = 10
initial_epochs=10
total_epochs =  initial_epochs + fine_tune_epochs

history_fine = model.fit(x_train,y_train,
                         epochs=total_epochs,
                         initial_epoch=initial_epochs,
                         validation_data=(x_test, y_test))

Epoch 11/20
1563/1563 [==============================] - 97s 22ms/step - loss: 0.2936 - accuracy: 0.8986 - val_loss: 1.6156 - val_accuracy: 0.6634
Epoch 12/20
1563/1563 [==============================] - 34s 22ms/step - loss: 0.1578 - accuracy: 0.9434 - val_loss: 1.7683 - val_accuracy: 0.6690
Epoch 13/20
1563/1563 [==============================] - 33s 21ms/step - loss: 0.1005 - accuracy: 0.9642 - val_loss: 1.9528 - val_accuracy: 0.6665
Epoch 14/20
1563/1563 [==============================] - 33s 21ms/step - loss: 0.0762 - accuracy: 0.9746 - val_loss: 2.1577 - val_accuracy: 0.6628
Epoch 15/20
1563/1563 [==============================] - 33s 21ms/step - loss: 0.0630 - accuracy: 0.9783 - val_loss: 2.2087 - val_accuracy: 0.6693
Epoch 16/20
1563/1563 [==============================] - 33s 21ms/step - loss: 0.0463 - accuracy: 0.9849 - val_loss: 2.4062 - val_accuracy: 0.6728
Epoch 17/20
1563/1563 [==============================] - 33s 21ms/step - loss: 0.0325 - accuracy: 0.9899 - val_loss: 2